# Simple Hartree-Fock SCF calculation: this progam uses stored integrals to perform a Hartree-Fock SCF calculation for two electron atoms and diatomic molecules. it also calculates the dipole moment.

# (1) Import the libraries needed to run the notebook

In [1]:
import numpy as np

# (2) Define function to symmetrize a matrix 

In [2]:
# Symmetrize a matrix given a triangular one
def symmetrise(Mat): 
    return Mat + Mat.T - np.diag(Mat.diagonal())

# (3) Define function returning compound index given four indices using Yoshimine sort (provides single index for the two-electron integrals)

In [3]:
# Accesing the correct two-electorn matrix element is not straightforward, particularly for systems wit many
#electrons

def eint(a,b,c,d): # Return compound index given four indices using Yoshimine sort
    if a > b: ab = a*(a+1)/2 + b
    else: ab = b*(b+1)/2 + a
    if c > d: cd = c*(c+1)/2 + d
    else: cd = d*(d+1)/2 + c
    if ab > cd: abcd = ab*(ab+1)/2 + cd
    else: abcd = cd*(cd+1)/2 + ab

    return abcd

# (4) Define function to return value of a two electron integral given the 4 indices of the basis functions

In [4]:
def tei(a, b, c, d): # Return value of two electron integral
    return twoe.get(eint(a, b, c, d), 0) 

# (5) Define function to make Fock Matrix

In [5]:
def makefock(Hcore, P, dim): # Make Fock Matrix
    F = np.zeros((dim, dim))
    for i in range(0, dim):
        for j in range(0, dim):
            F[i,j] = Hcore[i,j]
            for k in range(0, dim):
                for l in range(0, dim):
                    F[i,j] = F[i,j] + P[k,l]*(tei(i+1,j+1,k+1,l+1)-0.5*tei(i+1,k+1,j+1,l+1))
    
    return F 

# (6) Define function to generate Fock matrix in orthonormal AO basis

In [6]:
# Use the transformation matrix X to convert the original Fock matrix into one in the orthonormal basis
def fprime(X, F): 
    return np.dot(np.transpose(X), np.dot(F, X)) 


# (7) Define function to make density matrix and store the old one needed to test for convergence

In [7]:
def makedensity(C, D, dim, Nelec): 
    Dold = np.zeros((dim, dim))
    for mu in range(0, dim):
        for nu in range(0, dim):
            Dold[mu,nu] = D[mu, nu]
            D[mu,nu] = 0
            for m in range(0, int(Nelec/2)):
                D[mu,nu] = D[mu,nu] + 2*C[mu,m]*C[nu,m]

    return D, Dold 

# (8) Define function to calculate change in density matrix using Root Mean Square Deviation (RMSD)

In [8]:
# The function below sums the square of the difference between the each old and new element of the density matrix
# and then takes the square root
def deltap(D, Dold): 
    DELTA = 0.0
    for i in range(0, dim):
        for j in range(0, dim):
            DELTA = DELTA + ((D[i,j] - Dold[i,j])**2)

    return (DELTA)**(0.5)

# (9) Define function to calculate the energy at an iteration

In [9]:
# We use the expression of the energy in the Hartree Fock approach to determine the energy for an iteration
def currentenergy(D, Hcore, F, dim): 
    EN = 0
    for mu in range(0, dim):
        for nu in range(0, dim):
            EN += 0.5*D[mu,nu]*(Hcore[mu,nu] + F[mu,nu])
            
    EN=EN*convert_to_eV      # Convert energy to EV  
    return EN

# (10) Define function to calculate the dipole moment 

In [10]:
# We use the density matrix to evaluate the dipole moment for an iteration. This requires matrices
# involvinf z and the basis functions that will be read in before the function is called
def currentdipole(D, ZM, DIPNUC, dim): 
    DIPOLE = DIPNUC
    PROD=np.dot(ZM,D)
    for mu in range(0, dim):
        DIPOLE=DIPOLE+ PROD[mu,mu]
    DIPOLE=DIPOLE*convert_to_debye # Convert dipole moment to Debye
    return DIPOLE

# (11) Define some essential variables: number of electrons, number of basis functions and nuclei contribution to energy and dipole moment if needed 

In [11]:
# The calculations below use a set of units known as atomic units not introduced
# in the Module. The conversion factor needed to convert the resulting energies 
# to eV is:
convert_to_eV=27.21
#and to convert from atomic units of dipole moment to Debye
convert_to_debye=2.5916

Nelec = 2 # The number of electrons in our system

# Data for the specific physical system

# Nuclear data for He
DIPNUC=0.0 # This will not be used but it's best to set it to zero
ENUC=0.0  # There is no nuclear repulsion in the case of an atom

# Nuclear data for HeH+ in atomic units
#DISTNUC=1.582       # the bondlength of the molecule. 
#ENUC = 1.26429*convert_to_eV # the nuclear repulsion: 0 for atoms and  Z_1 x Z_2 / R for a diatomic molecules
#DIPNUC=-4*DISTNUC/5 + 2*DISTNUC/5

# Data for the basis set:dim is the number of basis functions in the set.
dim = 2

# Threshold to be used to decide on convergence
convergence_threshold=1.0e-7

# (12) Initilize the arrays to be used

In [12]:
# Initialize integrals, and put them in a Numpy array
T = np.zeros((dim, dim))
S = np.zeros((dim, dim)) 
V = np.zeros((dim, dim))
#ZM = np.zeros((dim, dim))
P = np.zeros((dim, dim)) # Initialize the density matrix P
TEI = np.zeros((dim, dim)) # Initialize two-electron integrals

# (13) Read  the  elements (integrals) of the overlap, kinetic, potential integral, zm and two-electron matrices for different systems 

In [13]:
# The matrices required are real symmetric, so element ij is equal to element ji. This means that the
# files to be read only contain the ij and ii elements (but not the ji),i.e. the upper triangle of the matrix.

# Uncomment the lines below to read the data for the Helium atom for the basis set with 2 functions
Sraw = np.genfromtxt('./s_He_2bf.dat',dtype=None)                    # Overlap matrix 
Traw = np.genfromtxt('./t_He_2bf.dat',dtype=None)                    # Kinetic energy matrix
Vraw = np.genfromtxt('./v_He_2bf.dat',dtype=None)                    # Potential energy matrix
TEI  = np.genfromtxt('./two_elec_int_He_2bf.dat')                    # Two electron integrals

# Uncomment the lines below to read the data for the Helium atom for the basis set with 3 functions

#Sraw = np.genfromtxt('./s_He_3bf.dat',dtype=None)                    # Overlap matrix  
#Traw = np.genfromtxt('./t_He_3bf.dat',dtype=None)                    # Kinetic energy matrix
#Vraw = np.genfromtxt('./v_He_3bf.dat',dtype=None)                    # Potential energy matrix
#TEI  = np.genfromtxt('./two_elec_int_He_3bf.dat')                    # Two electron integrals

# Uncomment the lines below to read the data for the HeH+ for the basis set with 2 functions

#Sraw = np.genfromtxt('./s_HeHp_2bf.dat',dtype=None)               # Overlap matrix 
#Traw = np.genfromtxt('./t_HeHp_2bf.dat',dtype=None)               # Kinetic energy matrix
#Vraw = np.genfromtxt('./v_HeHp_2bf.dat',dtype=None)               # Potential energy matrix
#ZMraw= np.genfromtxt('./zm_HeHp_2bf.dat',dtype=None)              # Expectation value of z in the atomic basis
#TEI  = np.genfromtxt('./two_elec_int_HeHp_2bf.dat')               # Two electron integrals

# Uncomment the lines below to read the data for the HeH+ for the basis set with 4 functions

#Sraw = np.genfromtxt('./s_HeHp_4bf.dat',dtype=None)               # Overlap matrix 
#Traw = np.genfromtxt('./t_HeHp_4bf.dat',dtype=None)               # Kinetic energy matrix
#Vraw = np.genfromtxt('./v_HeHp_4bf.dat',dtype=None)               # Potential energy matrix
#ZMraw= np.genfromtxt('./zm_HeHp_4bf.dat',dtype=None)              # Expectation value of z in the atomic basis
#TEI  = np.genfromtxt('./two_elec_int_HeHp_4bf.dat')               # Two electron integrals

#  (14) Use the elements read above to build the square matrices required

In [14]:
# We put the elements read  into a matrix. The line for ZMraw must be uncommented 
# when the dipole moment is calculated  and commented if it is not.

for i in Sraw: S[i[0]-1, i[1]-1] = i[2]  
for i in Traw: T[i[0]-1, i[1]-1] = i[2] 
for i in Vraw: V[i[0]-1, i[1]-1] = i[2] 
#for i in ZMraw: ZM[i[0]-1, i[1]-1] = i[2] 

# We build the whole matrix from the upper/lower triangle information provided

S    = symmetrise(S) # Flip the triangular matrix in the diagonal
V    = symmetrise(V) # Flip the triangular matrix in the diagonal
T    = symmetrise(T) # Flip the triangular matrix in the diagonal
#ZM    = symmetrise(ZM) # Flip the triangular matrix in the diagonal

# We  put the two electron integrals in a dictionary. If we want to set them to 
# zero, we comment line 19 and  uncomment line 20.

twoe         = {eint(row[0], row[1], row[2], row[3]) : row[4] for row in TEI} # Put in python dictionary
#twoe         = {eint(row[0], row[1], row[2], row[3]) : 0.0 for row in TEI} # Put in python dictionary

# (15) Form core Hamiltonian matrix  and transformation matrix

In [15]:
# Form core Hamiltonian matrix as sum of one electron kinetic energy, T and potential energy, V matrices

Hcore  = T + V

# Use the overlap matrix to generate the transformation matrix

SVAL, SVEC   = np.linalg.eigh(S) # Diagonalize overlap matrix 
SVAL_minhalf = (np.diag(SVAL**(-0.5))) # Find inverse square root of eigenvalues 
S_minhalf    = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC))) #Form the transformation matrix, S_minhalf

# (16) Perform the calculation until self-consistency

In [16]:
# We perform the calculation until the difference between density matrices in two subsequent steps is smaller 
# than the value provided by convergence_threshold

# First, we set some initial values for the iterations 
DELTA        = 1 # Set placeholder value for delta
count        = 0 # Count how many SCF cycles are done, N(SCF)


#Iterate until convergence criterion is met. Then exit loop and calculate properties of interest
while DELTA > convergence_threshold :
    count     += 1                             # Add one to the  counter
    F         = makefock(Hcore, P, dim)        # Calculate Fock matrix, F
    Fprime    = fprime(S_minhalf, F)           # Calculate transformed Fock matrix, F'
    E, Cprime = np.linalg.eigh(Fprime)         # Diagonalize F' matrix to obtain eigenvalues and eigenvectors
    C         = np.dot(S_minhalf, Cprime)      # 'Back transform' the coefficients into original basis using transformation matrix
    P, OLDP   = makedensity(C, P, dim, Nelec)  # Make density matrix
    DELTA     = deltap(P, OLDP)                # Test for convergence. 
    
# Print the value of the energy obtained in each step.
# We must add the nuclear repoulsion term, ENUC, to the energy
# We avoid printing the first iteration, where the Fock matrix is set equal to Hcore and the energy is bad
    if count > 1: print("E = {:.8f}, Iteration = {}".format(currentenergy(P, Hcore, F, dim) + ENUC, count))

# Print the final result, including the dipole moment if it is not zero
print ("Convergence threshold",convergence_threshold)
print("SCF procedure complete, TOTAL E = {:.8f} eV".format(currentenergy(P, Hcore, F, dim) + ENUC))
if DIPNUC !=0.0: print("Dipole moment= {:.8f} D".format(currentdipole(P, ZM,DIPNUC, dim)))

E = -74.66908775, Iteration = 2
E = -77.01899233, Iteration = 3
E = -77.15146065, Iteration = 4
E = -77.16006361, Iteration = 5
E = -77.16062807, Iteration = 6
E = -77.16066513, Iteration = 7
E = -77.16066756, Iteration = 8
Convergence threshold 1e-07
SCF procedure complete, TOTAL E = -77.16066756 eV
